載入資料

In [26]:
import pandas as pd

df_train = pd.read_csv('./dataset/train.csv')
df_test = pd.read_csv('./dataset/test.csv')

整理特征

In [27]:
import re
from bs4 import BeautifulSoup


def preprocessor(text):
    soup = BeautifulSoup(text, 'html.parser')

    # find title
    title = soup.body.h1.string.strip().lower()

    # find author
    article_info = soup.head.find('div', {'class': 'article-info'})
    author_name = article_info.find('span', {'class': 'author_name'})
    if author_name != None:
        author = author_name.get_text()
    elif article_info.span != None:
        author = article_info.span.string
    else:
        author = article_info.a.string

    # clean author
    author = re.sub('\s+', ' ', author.strip().lower())
    if author.startswith('by '):
        author = author[3:]
    author = re.sub('&.*;', '&', author.replace(' and ', ' & '))

    author_list = []
    if author.find(',') == -1:
        author_list = re.split('\s*&\s*', author)
    else:
        authors = re.split('\s*,\s*', author)
        if authors[-1].find('&') == -1 or len(authors[-1].split('&')[-1].strip().split()) > 3:
            author_list.append(authors[0])
        else:
            author_list += authors[:-1]
            author_list += re.split('\s*&\s*', authors[-1])
    author = ' '.join([re.sub('\s+', '_', a) for a in author_list])

    # find channel
    channel = soup.body.article['data-channel'].strip().lower()

    # find topic
    a_list = soup.body.find('footer', {'class': 'article-topics'}).find_all('a')
    topic_list = [a.string.strip().lower() for a in a_list]
    topic = ' '.join([re.sub('\s+', '_', t) for t in topic_list])

    # find datetime
    article_info = soup.head.find('div', {'class': 'article-info'})
    try:
        date_time = article_info.time['datetime']
    except:
        date_time = 'Wed, 10 Oct 2014 15:00:43'
    match_obj = re.search('([\w]+),\s+([\d]+)\s+([\w]+)\s+([\d]+)\s+([\d]+):([\d]+):([\d]+)', date_time)
    day, date, month, year, hour, minute, second = match_obj.groups()
    day, month = day.lower(), month.lower()

    # find content
    content = soup.body.find('section', {'class': 'article-content'}).get_text()
    content_len = len(content)

    # find see also
    num_see_also = len(re.findall('see also', content.lower()))

    # find image
    num_image = len(soup.body.find_all('img'))

    # find a
    num_a = len(soup.body.find_all('a'))

    return title, author, channel, topic, day, date, month, year, \
        hour, minute, second, content_len, num_see_also, num_image, num_a


feature_list = []
for text in df_train['Page content']:
    feature_list.append(preprocessor(text))
for text in df_test['Page content']:
    feature_list.append(preprocessor(text))

df_combine = pd.DataFrame(
    feature_list,
    columns=['Title', 'Author', 'Channel', 'Topic', 'Day', 'Date', 'Month', 'Year',
             'Hour', 'Minute', 'Second', 'Content_Len', 'Num_See_Also', 'Num_Image', 'Num_A']
)

<>:22: SyntaxWarning: invalid escape sequence '\s'
<>:29: SyntaxWarning: invalid escape sequence '\s'
<>:31: SyntaxWarning: invalid escape sequence '\s'
<>:36: SyntaxWarning: invalid escape sequence '\s'
<>:37: SyntaxWarning: invalid escape sequence '\s'
<>:45: SyntaxWarning: invalid escape sequence '\s'
<>:53: SyntaxWarning: invalid escape sequence '\w'
<>:22: SyntaxWarning: invalid escape sequence '\s'
<>:29: SyntaxWarning: invalid escape sequence '\s'
<>:31: SyntaxWarning: invalid escape sequence '\s'
<>:36: SyntaxWarning: invalid escape sequence '\s'
<>:37: SyntaxWarning: invalid escape sequence '\s'
<>:45: SyntaxWarning: invalid escape sequence '\s'
<>:53: SyntaxWarning: invalid escape sequence '\w'
C:\Users\11958\AppData\Local\Temp\ipykernel_36084\3966687669.py:22: SyntaxWarning: invalid escape sequence '\s'
  author = re.sub('\s+', ' ', author.strip().lower())
C:\Users\11958\AppData\Local\Temp\ipykernel_36084\3966687669.py:29: SyntaxWarning: invalid escape sequence '\s'
  author

打印前幾列

In [28]:
df_combine.head()

,Title,Author,Channel,Topic,Day,Date,Month,Year,Hour,Minute,Second,Content_Len,Num_See_Also,Num_Image,Num_A
0,nasa's grand challenge: stop asteroids from de...,clara_moskowitz,world,asteroid asteroids challenge earth space u.s. ...,wed,19,jun,2013,15,04,30,3591,4,1,21
1,google's new open source patent pledge: we won...,christina_warren,tech,apps_and_software google open_source opn_pledg...,thu,28,mar,2013,17,40,55,1843,1,1,16
2,ballin': 2014 nfl draft picks get to choose th...,sam_laird,entertainment,entertainment nfl nfl_draft sports television,wed,07,may,2014,19,15,20,6646,1,1,9
3,cameraperson fails deliver slapstick laughs,sam_laird,watercooler,sports video videos watercooler,fri,11,oct,2013,02,26,50,1821,1,0,11
4,nfl star helps young fan prove friendship with...,connor_finnegan,entertainment,entertainment instagram instagram_video nfl sp...,thu,17,apr,2014,03,31,43,8919,1,51,14


最後選擇

In [36]:
day_map = {'mon': 1, 'tue': 2, 'wed': 3,
           'thu': 4, 'fri': 5, 'sat': 6, 'sun': 7}
month_map = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
             'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}

df_copy = df_combine.copy()
df_copy['Day'] = df_copy['Day'].map(day_map)
df_copy['Month'] = df_copy['Month'].map(month_map)

df_copy = df_copy.drop(columns=['Title', 'Channel', 'Minute', 'Second', 'Num_See_Also', 'Num_Image', 'Num_A'])


In [30]:
df_copy.head()

,Author,Channel,Topic,Day,Date,Month,Year,Hour,Content_Len
0,clara_moskowitz,world,asteroid asteroids challenge earth space u.s. ...,3,19,6,2013,15,3591
1,christina_warren,tech,apps_and_software google open_source opn_pledg...,4,28,3,2013,17,1843
2,sam_laird,entertainment,entertainment nfl nfl_draft sports television,3,07,5,2014,19,6646
3,sam_laird,watercooler,sports video videos watercooler,5,11,10,2013,02,1821
4,connor_finnegan,entertainment,entertainment instagram instagram_video nfl sp...,4,17,4,2014,03,8919


文本處理

In [31]:
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('omw-1.4')


def tokenizer(text):
    if type(text) == np.ndarray:
        text = text[0]
    return re.split('\s+', text.strip())


def tokenizer_wnl(text):
    if type(text) == np.ndarray:
        text = text[0]
    text = re.sub("([\w]+)'[\w]+",
                  (lambda match_obj: match_obj.group(1)), text)
    text = re.sub('\.', '', text)
    text = re.sub('[^\w]+', ' ', text)
    wnl = WordNetLemmatizer()
    return [wnl.lemmatize(s) for s in re.split('\s+', text.strip())]

<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:18: SyntaxWarning: invalid escape sequence '\w'
<>:20: SyntaxWarning: invalid escape sequence '\.'
<>:21: SyntaxWarning: invalid escape sequence '\w'
<>:23: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:18: SyntaxWarning: invalid escape sequence '\w'
<>:20: SyntaxWarning: invalid escape sequence '\.'
<>:21: SyntaxWarning: invalid escape sequence '\w'
<>:23: SyntaxWarning: invalid escape sequence '\s'
C:\Users\11958\AppData\Local\Temp\ipykernel_36084\4085094747.py:12: SyntaxWarning: invalid escape sequence '\s'
  return re.split('\s+', text.strip())
C:\Users\11958\AppData\Local\Temp\ipykernel_36084\4085094747.py:18: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub("([\w]+)'[\w]+",
C:\Users\11958\AppData\Local\Temp\ipykernel_36084\4085094747.py:20: SyntaxWarning: invalid escape sequence '\.'
  text = re.sub('\.', '', text)
C:\Users\11958\AppData\Local\Temp\ipykernel_36084\

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer

# 先检查数据结构
print("df_copy columns:", df_copy.columns.tolist())
print("df_copy shape:", df_copy.shape)
print("Sample data:")
print("Author (col 0):", df_copy.iloc[0, 0])
print("Topic (col 1):", df_copy.iloc[0, 1])

# 确定数值列的索引（排除Author和Topic）
numeric_cols = [i for i, col in enumerate(df_copy.columns) if col not in ['Author', 'Topic']]
print("Numeric column indices:", numeric_cols)

trans_forest = ColumnTransformer(
    [('Author', CountVectorizer(tokenizer=tokenizer, lowercase=False), [0]),
     ('Topic', CountVectorizer(tokenizer=tokenizer_wnl, lowercase=False), [1]),
     ('numeric', 'passthrough', numeric_cols)],  # 明确指定数值列
    n_jobs=-1
)

trans_other = ColumnTransformer(
    [('Topic', CountVectorizer(tokenizer=tokenizer_wnl, lowercase=False), [1]),
     ('numeric', 'passthrough', numeric_cols)],  # 明确指定数值列，排除Author
    n_jobs=-1
)

訓練

In [33]:
from sklearn.model_selection import train_test_split

X_train_raw = df_copy.values[:df_train.shape[0]]
y_train_raw = (df_train['Popularity'].values == 1).astype(int)
X_test = df_copy.values[df_train.shape[0]:]

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_raw, y_train_raw, test_size=0.2, random_state=0)

In [34]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score


def training(clf):
    cv_results = cross_validate(clf, X_train_raw, y_train_raw,
                                scoring='roc_auc', return_train_score=True, return_estimator=True)
    print('train score: {:.5f} (+/-{:.5f})'.format(
        np.mean(cv_results['train_score']), np.std(cv_results['train_score'])))
    print('valid score: {:.5f} (+/-{:.5f})'.format(
        np.mean(cv_results['test_score']), np.std(cv_results['test_score'])))

    clf.fit(X_train, y_train)
    print('train score: {:.5f}'.format(roc_auc_score(
        y_train, clf.predict_proba(X_train)[:, 1])))
    print('valid score: {:.5f}'.format(roc_auc_score(
        y_valid, clf.predict_proba(X_valid)[:, 1])))
    return clf

LightGBM Classifier

In [37]:
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier

lgbm = Pipeline([('ct', trans_other),
                 ('clf', LGBMClassifier(random_state=0, learning_rate=0.009, n_estimators=300))])
lgbm = training(lgbm)

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\11958\.conda\envs\news_pred\Lib\site-packages\sklearn\compose\_column_transformer.py", line 1131, in _hstack
    check_array(X, accept_sparse=True, ensure_all_finite=False)
  File "c:\Users\11958\.conda\envs\news_pred\Lib\site-packages\sklearn\utils\validation.py", line 1053, in check_array
    array = _asarray_with_order(array, order=order, dtype=dtype, xp=xp)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\11958\.conda\envs\news_pred\Lib\site-packages\sklearn\utils\_array_api.py", line 757, in _asarray_with_order
    array = numpy.asarray(array, order=order, dtype=dtype)
           

Random Forest Classifier

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

forest = Pipeline([('ct', trans_forest),
                   ('clf', RandomForestClassifier(n_jobs=-1, random_state=0, n_estimators=300))])
forest = training(forest)

train score: 1.00000 (+/-0.00000)
valid score: 0.58552 (+/-0.00989)
train score: 1.00000
valid score: 0.58378


XGBoost Classifier

In [ ]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

xgboost = Pipeline([('ct', trans_other),
                    ('clf', XGBClassifier(verbosity=0, n_estimators=300))])
xgboost = training(xgboost)

train score: 0.81282 (+/-0.00500)
valid score: 0.58232 (+/-0.01309)
train score: 0.81618
valid score: 0.57805


CatBoost Classifier

In [ ]:
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier

catboost = Pipeline([('ct', trans_other),
                     ('clf', CatBoostClassifier(verbose=False, eval_metric='AUC', n_estimators=290, learning_rate=0.06))])
catboost = training(catboost)

train score: 0.68761 (+/-0.00276)
valid score: 0.59823 (+/-0.00827)
train score: 0.68671
valid score: 0.58746


投票

In [ ]:
from sklearn.ensemble import VotingClassifier
voting = VotingClassifier([('lgbm', lgbm), ('forest', forest), ('catboost', catboost)],
                          voting='soft', weights=[1, 0.05, 0.05])
voting = training(voting)

[LightGBM] [Info] Number of positive: 10906, number of negative: 11208
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011876 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1575
[LightGBM] [Info] Number of data points in the train set: 22114, number of used features: 596
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.493172 -> initscore=-0.027315
[LightGBM] [Info] Start training from score -0.027315


c:\Users\11958\.conda\envs\news_pred\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\11958\.conda\envs\news_pred\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 10905, number of negative: 11209
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027045 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1550
[LightGBM] [Info] Number of data points in the train set: 22114, number of used features: 584
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.493127 -> initscore=-0.027496
[LightGBM] [Info] Start training from score -0.027496


c:\Users\11958\.conda\envs\news_pred\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\11958\.conda\envs\news_pred\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 10905, number of negative: 11209
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020836 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1552
[LightGBM] [Info] Number of data points in the train set: 22114, number of used features: 587
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.493127 -> initscore=-0.027496
[LightGBM] [Info] Start training from score -0.027496


...

train score: 0.78434 (+/-0.00449)
valid score: 0.60368 (+/-0.00829)
[LightGBM] [Info] Number of positive: 10885, number of negative: 11229
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020727 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1567
[LightGBM] [Info] Number of data points in the train set: 22114, number of used features: 592
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.492222 -> initscore=-0.031114
[LightGBM] [Info] Start training from score -0.031114


c:\Users\11958\.conda\envs\news_pred\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


train score: 0.77807


c:\Users\11958\.conda\envs\news_pred\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


valid score: 0.59836


Testing Data Prediction

In [ ]:
best_model = voting

y_score = best_model.predict_proba(X_test)[:, 1]
df_pred = pd.DataFrame({'Id': df_test['Id'], 'Popularity': y_score})
df_pred.to_csv('submission_103.csv', index=False)

c:\Users\11958\.conda\envs\news_pred\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
